## Построение рекомендательной системы

1. [Weghted Popular Recommender](#1)
2. [Collaborative Filtering](#2) 
    - [Content Based Recommender](#2.1)
    - [User-Based Recommender](#2.2)
    - [Model-Based Recommender System](#2.3)
3. [Сравнение моделей](#3)

<b>Импорт библиотек и скриптов</b>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import pickle

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval

matplotlib.rcParams.update({'font.size': 12})
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate

from sklearn.metrics import r2_score, mean_squared_error

import warnings
warnings.filterwarnings("ignore")

sns.set_style('whitegrid')
sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)

<b>Загрузка данных</b>

In [2]:
movies_metadata = pd.read_csv('prepared_data/movies_metadata.csv')
credits = pd.read_csv('prepared_data/credits.csv')
keywords = pd.read_csv('prepared_data/keywords.csv')
links = pd.read_csv('full_data/links.csv')
ratings = pd.read_csv('full_data/ratings.csv')

In [3]:
genome_scores = pd.read_csv('full_data/genome-scores.csv')
genome_tags = pd.read_csv('full_data/genome-tags.csv')

In [4]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84851 entries, 0 to 84850
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tmdbId                     84851 non-null  int64  
 1   belongs_to_collection      84851 non-null  object 
 2   budget                     84851 non-null  int64  
 3   genres                     84851 non-null  object 
 4   original_language          84851 non-null  object 
 5   overview                   84077 non-null  object 
 6   popularity                 84851 non-null  int64  
 7   production_companies       84851 non-null  object 
 8   production_countries       84851 non-null  object 
 9   release_date               84851 non-null  object 
 10  revenue                    84851 non-null  int64  
 11  runtime                    84851 non-null  int64  
 12  spoken_languages           84851 non-null  object 
 13  status                     84851 non-null  obj

In [4]:
def memory_usage(df):
    return df.memory_usage(deep=True).sum()/1024**2
    
def downcast_numbers(df):
    selected_col_int = df.select_dtypes(include=['int']).columns
    selected_col_float = df.select_dtypes(include=['float']).columns
    
    memory_before = memory_usage(df[selected_col_int])+ memory_usage(df[selected_col_float])
    
    for col in selected_col_int:
        mx_col = df[col].max()
        mn_col = df[col].min()
        if mn_col > np.iinfo(np.int8).min and mx_col < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif mn_col > np.iinfo(np.int16).min and mx_col < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif mn_col > np.iinfo(np.int32).min and mx_col < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
    for col in selected_col_float:
        mx_col = df[col].max()
        mn_col = df[col].min()
        
        if mn_col > np.finfo(np.float16).min and mx_col < np.finfo(np.float16).max:
            df[col] = df[col].astype(np.float16)
        elif mn_col > np.finfo(np.float32).min and mx_col < np.finfo(np.float32).max:
            df[col] = df[col].astype(np.float32)
        else:
            df[col] = df[col].astype(np.float64)
    memory_after = memory_usage(df[selected_col_int])+ memory_usage(df[selected_col_float])
    print(f'before the number downcast:{memory_before:.2f} MB')
    print(f'after the number downcast:{memory_after:.2f} MB')
    return df

def downcast_object(df, columns:list):
    selected_col = columns
    print(f'before the object downcast:{memory_usage(df[columns]):.2f} MB')
    for col in selected_col:
        numbr_of_unique = df[col].nunique()
        numbr_total = df[col].shape[0]
        if numbr_of_unique / numbr_total < 0.5:
            df[col] = df[col].astype('category')
    print(f'before the object downcast:{memory_usage(df[columns]):.2f} MB')
    return df

In [5]:
print('--Ratings--')
ratings = downcast_numbers(ratings)
ratings.timestamp = [datetime.fromtimestamp(x) for x in ratings['timestamp']]

print('--Keywords--')
keywords = downcast_numbers(keywords)

print('--Links--')
links = downcast_numbers(links)

print('--Credits--')
credits = downcast_numbers(credits)
credits.actors = credits.actors.apply(literal_eval)

print('--Movies_metadata--')
movies_metadata = downcast_numbers(movies_metadata)
movies_metadata.release_date = pd.to_datetime(movies_metadata.release_date)

--Ratings--
before the number downcast:1032.48 MB
after the number downcast:451.71 MB
--Keywords--
before the number downcast:0.65 MB
after the number downcast:0.33 MB
--Links--
before the number downcast:1.98 MB
after the number downcast:0.99 MB
--Credits--
before the number downcast:1.96 MB
after the number downcast:0.65 MB
--Movies_metadata--
before the number downcast:10.36 MB
after the number downcast:3.24 MB


In [6]:
movies_metadata.past_days = movies_metadata.past_days.apply(lambda x : int(x.replace(' days', '')))

In [7]:
def clean_numeric(x):
    try:
        return float(x)
    except:
        return np.nan
    
movies_metadata.vote_average = movies_metadata.vote_average.apply(clean_numeric)

### Test train split

In [8]:
ratings = ratings.merge(links[['movieId', 'tmdbId']], on='movieId')
ratings.drop(ratings[ratings.tmdbId.isna()].index, inplace=True)

In [9]:
ratings.tmdbId = ratings.tmdbId.astype('int')

По данным на 2018 год, в среднем жители России тратят 13 часов в месяц на просмотр фильмов и сериалов, как дома, так и в кинотеатрах. С 2020 года это число стало увеличиваться. За тестовую часть я взяла 60 последних дней. Каждый алгоритм будет рекомендовать 10 фильмов. 

In [10]:
val_size = timedelta(days=60)
data_train = ratings[ratings['timestamp'] < ratings['timestamp'].max() - val_size]
data_test = ratings[ratings['timestamp'] > ratings['timestamp'].max() - val_size]
data_train.shape, data_test.shape

((33669213, 5), (158000, 5))

In [13]:
print('Процент новых пользователей в трейне')
len(set(data_test.userId.unique()) - set(data_train.userId.unique())) / len(data_test.userId.unique())

Процент новых пользователей в трейне


0.28667595171773447

In [14]:
print('Процент новых фильмов в трейне')
len(set(data_test.tmdbId.unique()) - set(data_train.tmdbId.unique())) / len(data_test.tmdbId.unique())

Процент новых фильмов в трейне


0.04160501864236255

В тесте больше 28% юзеров нет в трейн датафрейме. То есть мы столкнемся с проблемой холодного старта. Чтобы правильно оценить работу алгоритмов, нужно будет учесть этот момент и расчитать метрики отдельно на всем тесте и только на "теплых" пользователях.

In [11]:
cold_users = list(set(data_test.userId.unique()) - set(data_train.userId.unique()))

In [12]:
result = data_test.groupby('userId')['tmdbId'].unique().reset_index()
result.columns=['userId', 'actual']
result.head()

,userId,actual
0,22,"[11, 562, 1891, 603, 120, 2048, 1572, 241, 207..."
1,33,"[103, 500, 10315, 10243]"
2,203,"[13, 238, 27205, 335984]"
3,237,"[496243, 587792, 242582]"
4,251,"[857, 807, 680, 278, 238, 550, 27205, 68718, 1..."


### <a id="1">Weghted Popular Recommender</a>

Используется IMDB's weighted rating formula

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

Следующий шаг — определить подходящее значение для *m*, минимального количества голосов, которое необходимо указать в таблице. В качестве порогового значения мы будем использовать **95-й процентиль**. Другими словами, чтобы фильм попал в чарты, он должен набрать больше голосов, чем как минимум 95% фильмов в списке.

In [64]:
vote_counts = movies_metadata['vote_count']
vote_averages = movies_metadata['vote_average']
C = vote_averages.mean()
m = vote_counts.quantile(0.95)
C, m

(5.850294330281906, 896.0)

In [65]:
qualified = movies_metadata[(movies_metadata['vote_count'] >= m)][['tmdbId', 'title', 'year', 'vote_count', 
                                                                   'vote_average', 'popularity', 'genre_name',
                                                                  'past_days']]
qualified.shape

(4246, 8)

Существует 4239 фильмов, у которых количество голосов превышает 893. Также, средненное количество среднего рейтинга составило 5.8 из 10

In [66]:
def imdb_weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [67]:
qualified['imdb_weight'] = qualified.apply(imdb_weighted_rating, axis=1)

In [68]:
qualified.sort_values('imdb_weight', ascending=False).head(15)

,tmdbId,title,year,vote_count,vote_average,popularity,genre_name,past_days,imdb_weight
314,278,The Shawshank Redemption,1994,24976,8.703125,199,Unknown_genre,10539,8.604326
837,238,The Godfather,1972,18992,8.710938,141,"['Drama', 'Crime']",18767,8.582059
12168,155,The Dark Knight,2008,30960,8.515625,127,"['Drama', 'Action', 'Crime', 'Thriller']",5494,8.440658
522,424,Schindler's List,1993,14816,8.570312,81,Unknown_genre,10821,8.415199
1186,240,The Godfather Part II,1974,11464,8.601562,89,"['Drama', 'Crime']",17756,8.402118
292,680,Pulp Fiction,1994,26176,8.484375,78,Unknown_genre,10552,8.397195
5493,129,Spirited Away,2001,15128,8.539062,102,Unknown_genre,8047,8.388717
351,13,Forrest Gump,1994,25696,8.476562,98,Unknown_genre,10631,8.388072
59972,496243,Parasite,2019,16672,8.515625,92,Unknown_genre,1524,8.379688
7010,122,The Lord of the Rings: The Return of the King,2003,22608,8.476562,129,"['Adventure', 'Fantasy', 'Action']",7183,8.376446


В топ 15 больше всего входят такие фильмы, которые заработали свою популярность за многие года. Топовый фильм - "Побег уз Шоушенка" вообще снят практически 20 лет назад. Это признанные и, можно сказать, эталонные фильмы. Такой топ тоже имеет место быть, но совсем не дает шанса более новым достойным фильмам, которые в силу своего сравнитено короткого периода жизни на экранах, еще не получили достаточное количество голосов.

Поэтому сбалансируем веса с помощью признака past_days, который содержит количество дней, прошедших с момента релиза.

In [69]:
T = movies_metadata.past_days.max()
T

54291

In [70]:
def imdb_weighted_rating_improved(x):
    v = x['vote_count']
    R = x['vote_average']
    t = x['past_days']
    p = x['popularity']
    return (v/(v+m) * R) + (m/(m+v) * C) + ((T-t)/T)

In [71]:
qualified['imdb_weight_impr'] = qualified.apply(imdb_weighted_rating_improved, axis=1)

In [72]:
qualified.sort_values('imdb_weight_impr', ascending=False).head(15)

,tmdbId,title,year,vote_count,vote_average,popularity,genre_name,past_days,imdb_weight,imdb_weight_impr
314,278,The Shawshank Redemption,1994,24976,8.703125,199,Unknown_genre,10539,8.604326,9.410205
59972,496243,Parasite,2019,16672,8.515625,92,Unknown_genre,1524,8.379688,9.351617
12168,155,The Dark Knight,2008,30960,8.515625,127,"['Drama', 'Action', 'Crime', 'Thriller']",5494,8.440658,9.339463
20996,157336,Interstellar,2014,32960,8.421875,170,Unknown_genre,3191,8.353818,9.295042
41866,372058,Your Name.,2016,10528,8.507812,116,Unknown_genre,2531,8.299380,9.252761
7010,122,The Lord of the Rings: The Return of the King,2003,22608,8.476562,129,"['Adventure', 'Fantasy', 'Action']",7183,8.376446,9.244141
5493,129,Spirited Away,2001,15128,8.539062,102,Unknown_genre,8047,8.388717,9.240497
837,238,The Godfather,1972,18992,8.710938,141,"['Drama', 'Crime']",18767,8.582059,9.236385
56654,324857,Spider-Man: Into the Spider-Verse,2018,14256,8.406250,110,"['Action', 'Adventure', 'Animation', 'Science ...",1699,8.255106,9.223812
14849,27205,Inception,2010,34784,8.367188,154,Unknown_genre,4765,8.303983,9.216215


Мы добились желаемого эффекта, в топе появилось больше "свежих" фильмов.

Такой подход можно применить к фильмам конкретных жанров, то есть, например, получить топ популярных фильмов жанра комедии или ужасов

In [73]:
movies_metadata['genre_name'] = movies_metadata['genre_name'].replace("Unknown_genre", "[]").apply(literal_eval)
s = movies_metadata.apply(lambda x: pd.Series(x['genre_name']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = movies_metadata.drop('genres', axis=1).join(s)

In [74]:
def get_top_by_genre(genre, percentile=0.85, N=15):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df['vote_count']
    vote_averages = df['vote_average']
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    T = df.past_days.max()
    
    qualified = df[(df['vote_count'] >= m)][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'past_days']]
    
    qualified['weight'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C) + ((T - x['past_days'])/ T), 
                                          axis=1)
    qualified = qualified.sort_values('weight', ascending=False).head(N)
    return qualified

Давайте посмотрим топ фильмов жанра фэнтези

In [75]:
get_top_by_genre('Fantasy')

,title,year,vote_count,vote_average,popularity,past_days,weight
7010,The Lord of the Rings: The Return of the King,2003,22608,8.476562,129,7183,9.008490
4873,The Lord of the Rings: The Fellowship of the Ring,2001,23568,8.406250,106,7896,8.939361
5824,The Lord of the Rings: The Two Towers,2002,20496,8.398438,109,7531,8.907375
16613,Harry Potter and the Deathly Hallows: Part 2,2011,19376,8.101562,154,4403,8.718815
7719,Harry Potter and the Prisoner of Azkaban,2004,20240,8.015625,204,7001,8.589860
82466,Puss in Boots: The Last Wish,2022,6636,8.257812,232,237,8.581949
4776,Harry Potter and the Philosopher's Stone,2001,25648,7.914062,181,7928,8.523365
15442,Harry Potter and the Deathly Hallows: Part 1,2010,17952,7.800781,147,4640,8.441985
10367,Harry Potter and the Goblet of Fire,2005,19472,7.816406,161,6467,8.425401
6400,Pirates of the Caribbean: The Curse of the Bla...,2003,19424,7.796875,118,7328,8.386650


Напишем функцию рекомендаций

In [76]:
popular_df = qualified[['tmdbId', 'imdb_weight_impr']]
popular_df = popular_df.sort_values('imdb_weight_impr', ascending=False)

In [77]:
def weighted_popular_recommender(userId, N=5):
    return popular_df.head(N)['tmdbId'].to_list()

In [78]:
result['popular_recommendation'] = result.apply(lambda x: weighted_popular_recommender(x[0], 10), 1)
result.head()

,userId,actual,svd_recommender,svd_genome_recommender,popular_recommendation
0,22,"[11, 562, 1891, 603, 120, 2048, 1572, 241, 207...","[238, 14, 583, 670, 499556, 409926, 762, 240, ...","[670, 629, 14, 600, 515042, 496243, 20871, 77,...","[278, 496243, 155, 157336, 372058, 122, 129, 2..."
1,33,"[103, 500, 10315, 10243]","[192040, 367647, 122, 420714, 240, 200813, 325...","[240, 452830, 423, 489, 438631, 613, 269981, 3...","[278, 496243, 155, 157336, 372058, 122, 129, 2..."
2,203,"[13, 238, 27205, 335984]","[756498, 299536, 299534, 438449, 412098, 13477...","[497, 197, 857, 157336, 10191, 520594, 629, 49...","[278, 496243, 155, 157336, 372058, 122, 129, 2..."
3,237,"[496243, 587792, 242582]","[152044, 123778, 550416, 140465, 208988, 42071...","[520594, 367412, 10193, 146233, 496243, 324552...","[278, 496243, 155, 157336, 372058, 122, 129, 2..."
4,251,"[857, 807, 680, 278, 238, 550, 27205, 68718, 1...","[278, 32534, 192040, 496243, 420714, 238, 129,...","[278, 32534, 192040, 496243, 420714, 238, 129,...","[278, 496243, 155, 157336, 372058, 122, 129, 2..."


Расчитаем точность. Для этого напишем функцию precision_at_k

In [38]:
def precision_at_k(recommended_list, real_list, k=5):
    real_list = np.array(real_list)
    recommended_list = np.array(recommended_list)[:k]
    if len(recommended_list) ==0:
        return 0
    
    flags = np.isin(real_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [33]:
result.apply(lambda x: precision_at_k(x[2], x[1], 10), 1).mean()

0.09020427112349118

In [34]:
result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[2], x[1], 10), 1).mean()

0.01753986332574032

In [35]:
result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[2], x[1], 10), 1).mean()

0.2710121457489879

Все метрики будем записывать в отдельный датафрейм

In [19]:
metrics = pd.DataFrame({'full_precision': 0.09020427112349118, 
              'warm_precision': 0.01753986332574032, 'cold_precision': 0.2710121457489879}, index=['popular_recommender'])
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.01754,0.271012


Рекомендательная система, построеная на поиске топ популярных фильмов имеет свои плюсы и минусы. Из плюсов можно выделить следующие:
- простота расчета
- не имеет проблем с холодным стартом, а в данном случае показывает на них достаточно высокую метрику

Из минусов:
- обычно уступает другим алгоритмам на "теплых" пользователях
- для всех рекомендует одно и то же, не учитывая личные предпочтения пользователя
- не видит взаимосвязи между фильмами, если не произведен предпроцессинг (как жанр в примере выше)

Так как у данного подхода минусы достаточно критичны для нас, пойдем дальше и воспользуемся другим подходом построения рекомендательных систем, а именно - Коллаборативная фильтрация.

### <a id="2">Collaborative Filtering</a>

Решим вопрос взаимосвязи между фильмами и построем рекомендательную систему, основанную на метаданных фильма. 

Сначала подготовим выборку, так как мощности машины не хватит для расчета на всех датасете.

In [37]:
smm = movies_metadata.loc[(movies_metadata.year > 2000) | movies_metadata.tmdbId.isin(popular_df.tmdbId.to_list())]\
                        .sample(n=20000, random_state=42, replace=False, ignore_index=True)
sl = links[links.tmdbId.isin(smm.tmdbId.tolist())]
sc = credits[credits.tmdbId.isin(sl.tmdbId.tolist())]
sk = keywords[keywords.tmdbId.isin(sl.tmdbId.tolist())]
sr = data_train[data_train.movieId.isin(sl.movieId.tolist())]

### <a id="2.1">Content Based Recommender</a>

1. overview и tagline

In [38]:
smm['description'] = smm['overview'] + smm['tagline']
smm['description'] = smm['description'].fillna('')

In [39]:
tfidf = TfidfVectorizer(analyzer='word', 
                stop_words='english',
                ngram_range=(1,2) # unigrams and bigrams
               )
description_matrix = tfidf.fit_transform(smm['description'])

In [40]:
description_matrix.shape

(20000, 244315)

По умолчанию TfidfVectorizer использует l2 нормализацию, т.е. косинусное сходство между двумя векторами представляет собой их скалярное произведение. Поэтому можно использовать sklearn.linear_kernel вместо cosine_similarities, поскольку это намного быстрее.

In [41]:
description_sim = linear_kernel(description_matrix, description_matrix)

Теперь у нас есть матрица попарного косинусного сходства для всех фильмов в нашем наборе данных. Следующий шаг — написать функцию, которая возвращает N наиболее похожих фильмов на основе косинусного показателя сходства.

In [42]:
tmdbIds = smm['tmdbId']
indices = pd.Series(smm.index, index=smm['tmdbId'])

In [45]:
def get_similar_item(tmdbId, tmdbIds, indices, cosine_sim, N):
    idx = indices[tmdbId]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:N+1]
    movie_indices = [i[0] for i in sim_scores]
    score = [i[1] for i in sim_scores]
    return pd.DataFrame({'tmdbId': tmdbIds.iloc[movie_indices], 'score':score})

In [44]:
tmdbId = smm[smm.title=='Spider-Man 3']['tmdbId'].values[0]
rec = get_similar_item(tmdbId, tmdbIds, indices, description_sim, 10)
smm[['tmdbId','title', 'genre_name','collection_name', 'production_companies_name',
     'release_date']].merge(rec[['tmdbId', 'score']], on='tmdbId').sort_values('score', ascending=False)

,tmdbId,title,genre_name,collection_name,production_companies_name,release_date,score
7,558,Spider-Man 2,"[Action, Adventure, Fantasy]",Spider-Man Collection,"['Marvel Enterprises', 'Laura Ziskin Productio...",2004-06-25,0.142006
3,634649,Spider-Man: No Way Home,"[Action, Adventure, Science Fiction]",Spider-Man (Avengers) Collection,"['Marvel Studios', 'Pascal Pictures', 'Columbi...",2021-12-15,0.125993
5,6488,Arachnophobia,[],NaN,"['Amblin Entertainment', 'Hollywood Pictures',...",1990-07-20,0.098224
8,3432,Mr. Brooks,[],NaN,"['Relativity Media', 'Tig Productions', 'Eden ...",2007-06-01,0.092701
0,961323,Nimona,[],NaN,"['Annapurna Pictures', 'DNEG']",2023-06-23,0.087427
1,104950,Holy Musical B@man!,[],NaN,['Starkid Productions'],2012-03-22,0.064451
2,311585,Make Your Own Damn Movie!,[],NaN,[],2005-01-01,0.064224
4,18533,Bronson,[],NaN,"['Vertigo Films', 'Aramid Entertainment Fund',...",2008-10-09,0.061255
6,429617,Spider-Man: Far From Home,"[Action, Adventure, Science Fiction]",Spider-Man (Avengers) Collection,"['Marvel Studios', 'Pascal Pictures', 'Columbi...",2019-06-28,0.060859
9,50087,Pokémon: Zoroark - Master of Illusions,"[Family, Animation, Adventure, Fantasy]",Pokémon: Diamond and Pearl Collection,"['East Japan Marketing & Communications', 'OLM...",2010-07-10,0.057697


Алгоритм достаточно хорошо отработал. Он нашел другие фильмы про Человека паука, даже нашел фильм из той же франшизы. Но вот третий фильм про Арахнофобию - боязнь пауков, хоть и связан с пауками, но совсем не то, что могло бы понравиться любителям Человека паука.

Фильмы принадлежат разным франшизам, в них играют разные актеры и их снимали разные киностудии. И все это имеет весомое значение в предпочтениях пользователей, а значит нужно понимать на сколько тот или иной параметр важен в поиске похожего фильма. Поэтому воспользуемся более информативными фичами, такими как жанр, франшиза, актеры, ключевые слова, директор и компания.

2. Подключаем director, actors, keywords_name, production_companies_name, genre_name и collection_name

In [45]:
smm = smm.merge(sc[['tmdbId', 'director', 'actors']], on='tmdbId', how='left')
smm = smm.merge(sk[['tmdbId', 'keywords_name']], on='tmdbId', how='left')

In [46]:
smm['keywords_name'] = smm['keywords_name'].fillna('[]').apply(literal_eval)

Сделаем небольшой препроцессинг

In [47]:
s = smm.apply(lambda x: pd.Series(x['keywords_name']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [48]:
s = s.value_counts()
s

woman director            1540
independent film           407
murder                     394
biography                  324
based on novel or book     317
                          ... 
ordination                   1
postponed wedding            1
warhead                      1
self absorption              1
mayoral election             1
Name: keyword, Length: 14554, dtype: int64

Ключевые слова встречаются от одного до 1527 раз. Мы отфильтруем те, что встречаются единажды, так как они бесполезны в поиске схожих фильмов.

In [49]:
s = s[s > 1]

Также мы нормализуем слова, с помощью метода SnowballStemmer, чтобы привести слова к их изначальной форме

In [50]:
stemmer = SnowballStemmer('english')

In [51]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [52]:
smm['keywords_name'] = smm['keywords_name'].apply(filter_keywords)
smm['keywords_name'] = smm['keywords_name'].apply(lambda x: [stemmer.stem(i) for i in x])

Фильтрация и веса:
- возьмем только первых трех актеров и первую киностудию
- чтобы вес директора был больше, чем одного актера, продублируем его дважды 
- согласно позиции жанра в списке, продублируем первый жанр трижды, второй дважды
- коллекцию продублируем трижды

In [53]:
smm['soup'] = smm['keywords_name'].\
                apply(lambda x: [str.lower(i.replace(" ", "")) for i in x]) + smm['actors'].fillna('[]').\
                apply(lambda x: [str.lower(i.replace(" ", "")) for i in x[:3]]) + smm['director'].fillna('').astype('str').\
                apply(lambda x: str.lower(x.replace(" ", ""))).\
                apply(lambda x: [x,x]) + smm['genre_name'].\
                apply(lambda x: x[:1] * 3 + x[1:2] * 2 + x[2:]) + smm['collection_name'].astype('str').\
                apply(lambda x: str.lower(x.replace(" ", ""))).\
                apply(lambda x: [x,x,x]) + smm['production_companies_name'].\
                apply(literal_eval).apply(lambda x: [str.lower(i.replace(" ", "")) for i in x[:1]])
smm['soup'] = smm['soup'].apply(lambda x: ' '.join(x))

In [54]:
smm['soup'] = smm['description'] + ' ' + smm['soup']

In [55]:
soup_matrix = tfidf.fit_transform(smm['soup'])

In [56]:
soup_sim = linear_kernel(soup_matrix, soup_matrix)

In [57]:
rec = get_similar_item(tmdbId, tmdbIds, indices, soup_sim, 10)
smm[['tmdbId','title', 'genre_name','collection_name', 'actors', 'director','production_companies_name',
     'year']].merge(rec[['tmdbId', 'score']], on='tmdbId').sort_values('score', ascending=False)

,tmdbId,title,genre_name,collection_name,actors,director,production_companies_name,year,score
6,558,Spider-Man 2,"[Action, Adventure, Fantasy]",Spider-Man Collection,"[Tobey Maguire, Kirsten Dunst, James Franco, A...",Sam Raimi,"['Marvel Enterprises', 'Laura Ziskin Productio...",2004,0.454549
0,634649,Spider-Man: No Way Home,"[Action, Adventure, Science Fiction]",Spider-Man (Avengers) Collection,"[Tom Holland, Zendaya, Benedict Cumberbatch, J...",Jon Watts,"['Marvel Studios', 'Pascal Pictures', 'Columbi...",2021,0.194026
5,429617,Spider-Man: Far From Home,"[Action, Adventure, Science Fiction]",Spider-Man (Avengers) Collection,"[Tom Holland, Samuel L. Jackson, Jake Gyllenha...",Jon Watts,"['Marvel Studios', 'Pascal Pictures', 'Columbi...",2019,0.177714
2,6488,Arachnophobia,[],NaN,"[Jeff Daniels, Harley Jane Kozak, Garette Ratl...",Frank Marshall,"['Amblin Entertainment', 'Hollywood Pictures',...",1990,0.164025
4,513208,Itsy Bitsy,[],NaN,"[Bruce Davison, Denise Crosby, Elizabeth Rober...",Micah Gallo,"['Strange Vision', 'Paradox Film Group', 'Haci...",2019,0.105538
3,666750,Dragonheart: Vengeance,"[Fantasy, Action, Adventure]",DragonHeart Collection,"[Joseph Millson, Jack Kane, Arturo Muselli, He...",Ivan Silvestrini,['Universal Pictures'],2020,0.088022
8,345166,Emmanuelle Through Time: Rod Steele 0014 & Nak...,"[Fantasy, Action, Comedy]",Emmanuelle Through Time Collection,"[Allie Haze, Robert Donavan, Jason Sarcinelli,...",Rolfe Kanefsky,[],2012,0.083352
1,14208,The Librarian: Return to King Solomon's Mines,"[Fantasy, Action, Adventure]",The Librarian Collection,"[Noah Wyle, Gabrielle Anwar, Bob Newhart, Jane...",Jonathan Frakes,"['Turner Network Television', 'Electric Entert...",2006,0.078205
9,9986,Charlotte's Web,[],NaN,"[Julia Roberts, Steve Buscemi, John Cleese, Op...",Gary Winick,"['Paramount', 'Kerner Entertainment Company', ...",2006,0.077772
7,960700,Fullmetal Alchemist: The Revenge of Scar,"[Fantasy, Action, Adventure]",Fullmetal Alchemist (Live-Action) Collection,"[Ryosuke Yamada, Atomu Mizuishi, Tsubasa Honda...",Fumihiko Sori,"['Warner Bros. Japan', 'Oxybot']",2022,0.076290


Алгоритм стал более уверен в связи между фильмами одной коллекции, вес = 0.45. Плюс в списке появились другие фильмы из жанра фэнтези. 

3. genome

Есть еще один признак, через который можно определить взаимосвязь между фильмами - тэги. Так как информация емкая и весит мало, то можно произвести подчет по всему датасету.

In [47]:
genome_scores = genome_scores.merge(genome_tags, on='tagId')
genome_scores = genome_scores.merge(links[['movieId','tmdbId']], on='movieId')
genome_scores = genome_scores.merge(movies_metadata[['title','tmdbId']], on='tmdbId')

Создаем матрицу взаимоотношений тэга и фильма, где в ячейках вписываем показатель relevance

In [48]:
matrix_genome = pd.pivot_table(data=genome_scores,
                values='relevance',
               index = 'tmdbId',
               columns ='tag'
              )

In [49]:
genome_sim = cosine_similarity(matrix_genome, matrix_genome)

In [50]:
a = matrix_genome.reset_index()
tmdbIds_genome = a['tmdbId']
indices_genome = pd.Series(a.index, index=a['tmdbId'])
del a

In [62]:
rec = get_similar_item(tmdbId, tmdbIds_genome, indices_genome, genome_sim, 15)
movies_metadata[['tmdbId','title', 'genre_name','collection_name','production_companies_name',
     'year']].merge(rec[['tmdbId', 'score']], on='tmdbId').sort_values('score', ascending=False)

,tmdbId,title,genre_name,collection_name,production_companies_name,year,score
13,102382,The Amazing Spider-Man 2,"[Action, Adventure, Fantasy]",The Amazing Spider-Man Collection,"['Marvel Entertainment', 'Columbia Pictures', ...",2014,0.942401
6,1452,Superman Returns,"[Science Fiction, Action, Adventure]",Superman Collection,"['DC Comics', 'Legendary Pictures', 'Bad Hat H...",2006,0.937537
4,9738,Fantastic Four,"[Action, Adventure, Fantasy, Science Fiction]",Fantastic Four Collection,['Kumar Mobiliengesellschaft mbH & Co. Projekt...,2005,0.935019
5,36668,X-Men: The Last Stand,"[Adventure, Action, Science Fiction, Thriller]",X-Men Collection,"[""The Donners' Company"", '20th Century Fox']",2006,0.934384
12,1930,The Amazing Spider-Man,"[Action, Adventure, Fantasy]",The Amazing Spider-Man Collection,"['Marvel Entertainment', 'Laura Ziskin Product...",2012,0.933807
9,1724,The Incredible Hulk,[],NaN,"['Marvel Studios', 'Valhalla Motion Pictures']",2008,0.929044
3,558,Spider-Man 2,"[Action, Adventure, Fantasy]",Spider-Man Collection,"['Marvel Enterprises', 'Laura Ziskin Productio...",2004,0.929022
11,2080,X-Men Origins: Wolverine,"[Adventure, Action, Thriller, Science Fiction]",The Wolverine Collection,"[""The Donners' Company"", 'Seed Productions', '...",2009,0.928197
1,9480,Daredevil,[],NaN,"['Marvel Enterprises', 'New Regency Pictures',...",2003,0.926527
8,1979,Fantastic Four: Rise of the Silver Surfer,"[Adventure, Fantasy, Action, Thriller]",Fantastic Four Collection,"['1492 Pictures', 'Bernd Eichinger Productions...",2007,0.924260


Результат выглядит достаточно убедительно. Все фильмы из топ про супергероев, нет странных рекомендаций по типу Арахнофобия. 

Напишем функцию рекомендаций по наиболее похожим фильмам из просмотренных и получивших оценку выше 3.5. Такая фильтрация позволит отобрать фильмы согласно предпочтениям каждого пользователя.

In [20]:
users_data_train = data_train.loc[data_train.rating > 3.5].groupby('userId')['tmdbId'].unique().reset_index()
users_data_train.columns = ['userId', 'items']
users_data_train.head(2)

,userId,items
0,1,"[862, 197, 8839, 11, 13, 10895, 562, 31530, 18..."
1,2,"[862, 197, 13, 949, 4584, 8012, 9598, 687, 807..."


In [83]:
def content_recommender(userId, tmdbIds, indices, cosine_sim, N):
    recs = pd.DataFrame(columns=['tmdbId', 'score'])
    if users_data_train[users_data_train.userId==userId].shape[0] != 0:
        watched_list = users_data_train[users_data_train.userId==userId]['items'].values[0]
        if len(watched_list) > 10:
            rec_N = 2
        elif len(watched_list) > 2:
            rec_N = N // 2
        else:
            rec_N = N
        for i in watched_list:
            if i in indices.index:
                recs = pd.concat([recs, get_similar_item(i, tmdbIds, indices, cosine_sim, rec_N)], ignore_index=True)
                recs = recs.iloc[recs.sort_values('score').tmdbId.drop_duplicates().index]
        recs['tmdbId'] = recs['tmdbId'].astype('int')
        recs = recs.loc[~recs.tmdbId.isin(watched_list)]
        return recs.sort_values('score', ascending=False).tmdbId.to_list()[:N]
    else:
        return []

In [84]:
content_recommender(22, tmdbIds_genome, indices_genome, genome_sim, 10)

[120, 1891, 1894, 12444, 11, 767, 395991, 330459, 76338, 1893]

In [85]:
%%time
result['content_describtion_recommender'] = result.apply(lambda x: content_recommender(x[0], 
                                                                                       tmdbIds, indices, 
                                                                                       description_sim, 10), 1)

CPU times: total: 29min 29s
Wall time: 29min 31s


In [86]:
%%time
result['content_soup_recommender'] = result.apply(lambda x: content_recommender(x[0], 
                                                                                tmdbIds, indices, 
                                                                                soup_sim, 10), 1)

CPU times: total: 40min 54s
Wall time: 40min 55s


In [87]:
%%time
result['content_genome_recommender'] = result.apply(lambda x: content_recommender(x[0], 
                                                                                tmdbIds_genome, indices_genome, 
                                                                                genome_sim, 10), 1)

CPU times: total: 1h 42min 32s
Wall time: 1h 42min 35s


In [88]:
result.head(2)

,userId,actual,popular_recommendation,content_describtion_recommender,content_soup_recommender,content_genome_recommender,content_genome_recommender_improved
0,22,"[11, 562, 1891, 603, 120, 2048, 1572, 241, 207...","[278, 496243, 155, 157336, 372058, 122, 129, 2...","[352186, 60463, 335984, 607, 441829, 2048, 497...","[36658, 161, 275442, 607, 119450, 673, 274479,...","[120, 1891, 1894, 12444, 11, 767, 395991, 3304...","[496243, 157336, 27205, 324857, 769, 1891, 687..."
1,33,"[103, 500, 10315, 10243]","[278, 496243, 155, 157336, 372058, 122, 129, 2...","[458156, 249164, 71325, 433499, 9294, 761116, ...","[222715, 240, 603692, 458156, 844, 242, 843, 8...","[426426, 240, 674324, 359940, 423, 823754, 438...","[680, 550, 240, 637, 807, 423, 475557, 4935, 4..."


In [89]:
metrics.loc['content_describtion_recommender'] = [result.apply(lambda x: precision_at_k(x[3], x[1], 10), 1).mean(),
                                              result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[3], x[1], 10), 1).mean(),
                                              result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[3], x[1], 10), 1).mean()]
metrics.loc['content_soup_recommender'] = [result.apply(lambda x: precision_at_k(x[4], x[1], 10), 1).mean(),
                                       result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[4], x[1], 10), 1).mean(),
                                       result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[4], x[1], 10), 1).mean()]
metrics.loc['content_genome_recommender'] = [result.apply(lambda x: precision_at_k(x[5], x[1], 10), 1).mean(),
                                         result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[5], x[1], 10), 1).mean(),
                                         result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[5], x[1], 10), 1).mean()]

In [26]:
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.017540,0.271012
content_describtion_recommender,0.005014,0.007029,0.000000
content_soup_recommender,0.009655,0.013535,0.000000
content_genome_recommender,0.019762,0.027704,0.000000


Возможно можно улучшить работу контентных алгоритмов, воспользовавшись формулой взвешивания.

Сначала мы получим топ 3*N схожих фильмов, найдем их в списке самых популярных, усредним коэффициенты схожести и популярности и выведем топ N

In [91]:
def get_similar_item_improved(tmdbId,tmdbIds, indices, cosine_sim, N):
    idx = indices[tmdbId]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:3*N]
    movie_indices = [i[0] for i in sim_scores]
    score = [i[1] for i in sim_scores]
    result = pd.DataFrame({'tmdbId': tmdbIds.iloc[movie_indices], 'score_similar':score})
    
    movie_tmdbIds = tmdbIds.iloc[movie_indices]
    result = result.merge(popular_df, on='tmdbId',how='left')
    
    result['score'] = result.apply(lambda x : (x['imdb_weight_impr'] + x['score_similar'])/2, 1)
    result = result.sort_values('score', ascending=False)
    result['tmdbId'] = result['tmdbId'].astype('int')
    return result[['tmdbId', 'score']].head(N)

In [92]:
get_similar_item_improved(22 ,tmdbIds_genome, indices_genome, genome_sim, 10)

,tmdbId,score
28,120,5.002845
27,121,4.995937
11,315162,4.911410
18,527774,4.733094
20,502356,4.690530
6,85,4.680624
16,808,4.664852
22,166428,4.658531
14,89,4.648847
24,508439,4.643940


In [93]:
def content_recommender_improved(userId, tmdbIds, indices, cosine_sim, N):
    recs = pd.DataFrame(columns=['tmdbId', 'score'])
    if users_data_train[users_data_train.userId==userId].shape[0] != 0:
        watched_list = users_data_train[users_data_train.userId==userId]['items'].values[0]
        if len(watched_list) > 10:
            rec_N = 2
        elif len(watched_list) > 2:
            rec_N = N // 2
        else:
            rec_N = N
        for i in watched_list:
            if i in indices.index:
                recs = pd.concat([recs, get_similar_item_improved(i, tmdbIds, indices, cosine_sim, rec_N)], ignore_index=True)
                recs = recs.iloc[recs.sort_values('score').tmdbId.drop_duplicates().index]
        recs['tmdbId'] = recs['tmdbId'].astype('int')
        recs = recs.loc[~recs.tmdbId.isin(watched_list)]
        return recs.sort_values('score', ascending=False).tmdbId.to_list()[:N]
    else:
        return []

In [94]:
content_recommender_improved(22, tmdbIds_genome, indices_genome, genome_sim, 10)

[496243, 157336, 120, 324857, 769, 475557, 1891, 68718, 490132, 274]

In [95]:
%%time
result['content_genome_recommender_improved'] = result.apply(lambda x: content_recommender_improved(x[0], 
                                                                                tmdbIds_genome, indices_genome, 
                                                                                genome_sim, 10), 1)

CPU times: total: 2h 29min 46s
Wall time: 2h 29min 49s


In [96]:
metrics.loc['content_genome_recommender_improved'] = [result.apply(lambda x: precision_at_k(x[6], x[1], 10), 1).mean(),
                                                  result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[6], x[1], 10), 1).mean(),
                                                  result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x[6], x[1], 10), 1).mean()]
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.017540,0.271012
content_describtion_recommender,0.005014,0.007029,0.000000
content_soup_recommender,0.009655,0.013535,0.000000
content_genome_recommender,0.019762,0.027704,0.000000
content_genome_recommender_improved,0.025958,0.036391,0.000000


Точность алгоритма увеличилась на 0.6% на всем тесте и на 0.9% на темплых пользователях. 

По итогу content_genome_recommender_improved дал лучшие показатели точности

Так как контентные алгоритмы не могут работать с “холодными пользователями”, то на них применим popular recommender и получим следующий результат

In [97]:
def content_popular_recommender(userId, tmdbIds, indices, cosine_sim, N):
    recs = pd.DataFrame(columns=['tmdbId', 'score'])
    if users_data_train[users_data_train.userId==userId].shape[0] != 0:
        watched_list = users_data_train[users_data_train.userId==userId]['items'].values[0]
        for i in watched_list:
            if i in indices.index:
                recs = pd.concat([recs, get_similar_item_improved(i, tmdbIds, indices, cosine_sim, 1)], ignore_index=True)
                recs = recs.iloc[recs.sort_values('score').tmdbId.drop_duplicates().index]
        recs['tmdbId'] = recs['tmdbId'].astype('int')
        recs = recs.loc[~recs.tmdbId.isin(watched_list)]
        recommendations = recs.sort_values('score', ascending=False).tmdbId.to_list()
        if len(recommendations) < N:
            recommendations.extend(weighted_popular_recommender(userId, N-len(recommendations)))
        return recommendations[:N]
    else:
        return weighted_popular_recommender(userId, N)

In [98]:
%%time
result['content_genome_popular_recommender'] = result.apply(lambda x: content_popular_recommender(x[0], 
                                                                                tmdbIds_genome, indices_genome, 
                                                                                genome_sim, 10), 1)

CPU times: total: 2h 25min 9s
Wall time: 2h 25min 11s


In [99]:
metrics.loc['content_genome_popular_recommender'] = [result.apply(lambda x: precision_at_k(x.content_genome_popular_recommender, x[1], 10), 1).mean(),
                                                     result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x.content_genome_popular_recommender, x[1], 10), 1).mean(),
                                                     result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x.content_genome_popular_recommender, x[1], 10), 1).mean()]
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.017540,0.271012
content_describtion_recommender,0.005014,0.007029,0.000000
content_soup_recommender,0.009655,0.013535,0.000000
content_genome_recommender,0.019762,0.027704,0.000000
content_genome_recommender_improved,0.025958,0.036391,0.000000
content_genome_popular_recommender,0.104898,0.038139,0.271012


Результат стал гораздо лучше (на 8% на тесте и на 0.2% для теплых пользователей)

### <a id="2.2"> User-based recommender</a>

Построим алгоритм, который работает на основе схожести предпочтений пользователей. Если один пользователь поставил трем фильмам высшую оценку, а другой пользователь поставил высшую оченку двум из этих фильмов, то есть вероятность, что ему понравится и третий фильм.

Сначала посмотрим сколько оценок заработал каждый из фильмов

In [100]:
values_pd = sr["tmdbId"].value_counts() 
values_pd

278       121752
603       106552
550        85759
329        82861
862        76603
           ...  
280127         1
333657         1
103903         1
537050         1
354148         1
Name: tmdbId, Length: 19418, dtype: int64

Фильмы, которым поставили менее 100 оценок посчитаем за редкие. Я не буду их включать в алгоритм

In [101]:
sr.rating = sr.rating.apply(clean_numeric)
sr.rating.mean()

3.5837156303149422

In [102]:
rare_movies = values_pd[values_pd < 100].index
df = sr[~sr["tmdbId"].isin(rare_movies)]

Приводим датафрейм к нужному формату

In [103]:
df = df[['userId', 'rating', 'tmdbId']]

In [104]:
df.rating = df.rating.astype('float64')
df.userId = df.userId.astype('int64')
df.tmdbId = df.tmdbId.astype('int64')

Создаем матрицу

In [105]:
user_tmbdId_matrix = df.groupby(["userId","tmdbId"])["rating"].mean().unstack()

In [106]:
user_tmbdId_matrix.shape

(318286, 2884)

Сначала посмотрим логику алгоритма на рандомном пользователе

In [107]:
sample_guy = user_tmbdId_matrix.sample(1,random_state=30).index[0]
sample_guy

214196

In [108]:
random_user_df = user_tmbdId_matrix.loc[user_tmbdId_matrix.index == sample_guy]
random_user_df

tmdbId,5,12,14,18,24,27,33,38,67,69,...,807196,819876,862965,866413,869626,882598,894205,899112,937278,949423
userId,,,,,,,,,,,,,,,,,,,,,
214196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
movies_watched = random_user_df.dropna(axis=1).columns.tolist() #  те фильмы, которые посмотрел пользователь
len(movies_watched)

22

In [110]:
movies_watched_df = user_tmbdId_matrix[movies_watched] # колонки фильмов

In [111]:
user_movie_count = movies_watched_df.notnull().sum(axis=1) # сколько из просмотренных фильмов, просмотрели другие пол-ли

In [112]:
user_movie_count.max() # есть те, кто просмотрели все фильмы

22

Найдем тем пользователей, которые посмотрели хотя бы 80% из фильмов

In [113]:
users_same_movies = user_movie_count[user_movie_count > (len(movies_watched) * 0.8)].index
users_same_movies

Int64Index([   897,   1917,   2327,   4127,   5283,   6386,   7010,   7155,
              8558,   8774,
            ...
            317947, 318296, 318475, 319046, 319924, 321032, 324914, 327439,
            329594, 330842],
           dtype='int64', name='userId', length=384)

In [114]:
filted_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]

Посчитаем корреляцию между пользователями

In [115]:
corr_df = filted_df.T.corr().unstack().drop_duplicates()

In [116]:
corr_df[sample_guy][corr_df[sample_guy] > 0.10].sort_values(ascending=False)[:15]

userId
295694    0.746960
310438    0.688422
296547    0.634497
309284    0.597806
240023    0.582425
312288    0.576570
293779    0.575886
314721    0.568432
283685    0.564769
224971    0.544949
318296    0.540210
215551    0.524618
311829    0.491449
302776    0.469530
298775    0.468805
dtype: float64

Посмотрим у с какими пользователями корреляция с нашим рандомным человеком превышает 30%

In [117]:
top_users = pd.DataFrame(corr_df[sample_guy][corr_df[sample_guy] > 0.10].sort_values(ascending=False)[:15], columns=["corr"])
top_users

,corr
userId,
295694,0.746960
310438,0.688422
296547,0.634497
309284,0.597806
240023,0.582425
312288,0.576570
293779,0.575886
314721,0.568432
283685,0.564769


Создадим датасет с этими юзерами и их оценками

In [118]:
top_users_ratings = pd.merge(top_users, ratings[["userId", "tmdbId", "rating"]], how='inner', on="userId")

top_users_ratings

,userId,corr,tmdbId,rating
0,295694,0.746960,862,4.0
1,295694,0.746960,197,5.0
2,295694,0.746960,11,4.5
3,295694,0.746960,13,5.0
4,295694,0.746960,10449,3.5
...,...,...,...,...
34102,298775,0.468805,321989,3.0
34103,298775,0.468805,721857,3.5
34104,298775,0.468805,545330,3.5
34105,298775,0.468805,537946,3.5


Взвесим их оценки с помощью коэффициента схожести и выведем топ 10

In [119]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

In [120]:
recommendation_df = top_users_ratings.pivot_table(values="weighted_rating", index="tmdbId", aggfunc="mean")
recommendation_df

,weighted_rating
tmdbId,
3,1.268993
5,1.597190
6,2.190369
11,2.311569
12,2.337883
...,...
1077280,2.017993
1077814,1.729709
1087040,2.017993


In [121]:
recommendation_df.sort_values("weighted_rating" , ascending=False).head(10)

,weighted_rating
tmdbId,
378064,3.442109
10228,3.442109
20996,3.361320
756688,3.361320
11304,3.361320
23495,3.361320
158,3.172483
20873,3.172483
20993,3.172483


In [122]:
def user_based_recommender(userId, N):
    try:
        if user_tmbdId_matrix.loc[user_tmbdId_matrix.index == userId].shape[0] == 0:
            return weighted_popular_recommender(userId, N)
        watched_list = user_tmbdId_matrix.loc[user_tmbdId_matrix.index == userId].dropna(axis=1).columns.tolist()
        watched_df = user_tmbdId_matrix[watched_list]
        user_movie_count = watched_df.notnull().sum(axis=1)
        users_same_movies = user_movie_count[user_movie_count > (len(watched_list) * 0.8)].index
        if len(users_same_movies) == 0:
            return weighted_popular_recommender(userId, N)
        filted_df = watched_df[watched_df.index.isin(users_same_movies)]
        corr_df = filted_df.T.corr().unstack().drop_duplicates()
        top_users = pd.DataFrame(corr_df[userId][corr_df[userId] > 0.1].sort_values(ascending=False)[:15], columns=["corr"])
        top_users_ratings = pd.merge(top_users, ratings[["userId", "tmdbId", "rating"]], how='inner', on="userId")
        top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
        recommendation_df = top_users_ratings.pivot_table(values="weighted_rating", index="tmdbId", aggfunc="mean")
        return recommendation_df.sort_values("weighted_rating" , ascending=False).head(N).index.to_list()
    except Exception as e:
        print(e)
        return []

In [123]:
user_based_recommender(sample_guy, 10)

[378064, 10228, 20996, 756688, 11304, 23495, 158, 20873, 20993, 20878]

К сожалению, ресурсов машины не хватает для того, чтобы расчитать рекомендации для всех в датасете.

### <a id="2.3">Model-Based Recommender System</a>

С помощью алгоритмов выше мы рекомендовали фильмы, которые могут понравиться пользователям, но есть возможность спрогнозировать оценку пользователей к каждому фильму. И сделаем мы это с помощью модели SVD.

In [13]:
reader = Reader(rating_scale=(0.5, 5))

In [14]:
data = Dataset.load_from_df(data_train[['userId', 'tmdbId', 'rating']], reader)

In [15]:
trainset, testset = train_test_split(data, test_size=.25)
svd_model = SVD(random_state=42)
svd_model.fit(trainset)
predictions = svd_model.test(testset)

In [16]:
accuracy.rmse(predictions)

RMSE: 0.7896


0.7895724129978947

Ошибка RMSE составляет 0.7896, чтобы понять насколько это критично, посмотрим на пример

In [56]:
svd_model.predict(uid=318248, iid=1832, verbose=True)
svd_model.predict(uid=318248, iid=4247, verbose=True)

user: 318248     item: 1832       r_ui = None   est = 3.95   {'was_impossible': False}
user: 318248     item: 4247       r_ui = None   est = 3.52   {'was_impossible': False}


Prediction(uid=318248, iid=4247, r_ui=None, est=3.5232360100754208, details={'was_impossible': False})

In [57]:
data_train[(data_train["userId"] == 318248) & (data_train['tmdbId'].isin([4247, 1832]))]

,userId,movieId,rating,timestamp,tmdbId
15029319,318248,3052,4.5,2005-03-07 08:46:01,1832
16217305,318248,3785,3.5,2005-03-07 09:00:03,4247


Видно, что реальная оценка, которую поставил пользователь, и предсказанная отличаются. В первом примере алгоритм ошибся на 0.55, а во втором всего на 0.02. На рекомендации это не сильно повлияет, если более подходящий под предпочтения фильм получал больше, чем менее предпочтительный.

In [35]:
def svd_recommender(user_id, N):
    didnt_watch = list(set(data_train["tmdbId"].unique()) - set(data_train.loc[data_train['userId']==user_id,'tmdbId'].to_list()))
    temp_dict={}
    
    for i in didnt_watch:
        
        temp_dict[i] = svd_model.predict(uid=user_id, iid=i)[3]
     
    suggestions = pd.DataFrame(temp_dict.items(),columns=["tmdbId",'possible_rate']).sort_values(by="possible_rate", ascending=False).head(N)
    
    return suggestions.tmdbId.to_list()

In [36]:
result['svd_recommender'] = result.apply(lambda x: svd_recommender(x.userId, 10), 1)

In [39]:
metrics.loc['svd_recommender'] = [result.apply(lambda x: precision_at_k(x.svd_recommender, x[1], 10), 1).mean(),
                                  result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x.svd_recommender, x[1], 10), 1).mean(),
                                  result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x.svd_recommender, x[1], 10), 1).mean()]
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.017540,0.271012
content_describtion_recommender,0.005014,0.007029,0.000000
content_soup_recommender,0.009655,0.013535,0.000000
content_genome_recommender,0.019762,0.027704,0.000000
content_genome_recommender_improved,0.033660,0.009860,0.092881
content_genome_popular_recommender,0.104898,0.038139,0.271012
svd_recommender,0.053018,0.018223,0.139595


Посчитаем точность R2 и MSE

In [40]:
data_test['predict'] = data_test.apply(lambda x: svd_model.predict(uid=x.userId, iid=x.tmdbId, verbose=False)[3], 1)

In [41]:
r2_score(data_test.rating.tolist(), data_test.predict.to_list())

0.20766001720362315

In [42]:
mean_squared_error(data_test.rating.tolist(), data_test.predict.to_list())

0.9139608207386182

Можно выбрать более специфичный сцинарий применения и немного видоизменить алгоритм. Когда определенный пользователь смотрит фильм, то на странице внизу можно отобразить схожие фильмы с просматриваемым и проранжировать их с точки зрения предсказанной оценки. 

In [43]:
def svd_genome(userId, tmdbId, N):
    similar_movies = get_similar_item(tmdbId, tmdbIds_genome, indices_genome, genome_sim, N*4)
    similar_movies['tmdbId'] = similar_movies['tmdbId'].astype('int')
    similar_movies['rating'] = similar_movies.apply(lambda x: svd_model.predict(userId, x['tmdbId'])[3], 1)
    similar_movies['final_score'] = similar_movies['rating'] * similar_movies['score']
    similar_movies = similar_movies.sort_values('final_score', ascending=False)
    return similar_movies[['tmdbId', 'final_score']].head(N)

In [51]:
svd_genome(12, 89, 2)

,tmdbId,final_score
48,85,3.806568
50,87,3.684315


In [52]:
def svd_genome_recommender(userId, N):
    recs = pd.DataFrame(columns=['tmdbId', 'final_score'])
    if users_data_train[users_data_train.userId==userId].shape[0] != 0:
        watched_list = users_data_train[users_data_train.userId==userId]['items'].values[0]
        for i in watched_list:
            if i in indices_genome.index:
                recs = pd.concat([recs, svd_genome(userId, i, 1)], ignore_index=True)
                recs = recs.iloc[recs.sort_values('final_score').tmdbId.drop_duplicates().index]
        recs['tmdbId'] = recs['tmdbId'].astype('int')
        recs = recs.loc[~recs.tmdbId.isin(watched_list)]
        recommendations = recs.sort_values('final_score', ascending=False).tmdbId.to_list()
        if len(recommendations) < N:
            recommendations.extend(svd_recommender(userId, N))
        return recommendations[:N]
    else:
        return svd_recommender(userId, N)

In [53]:
svd_genome_recommender(12, 2)

[1422, 9631]

In [54]:
result['svd_genome_recommender'] = result.apply(lambda x: svd_genome_recommender(x.userId, 10), 1)

In [55]:
metrics.loc['svd_genome_recommender'] = [result.apply(lambda x: precision_at_k(x.svd_genome_recommender, x[1], 10), 1).mean(),
                                         result.loc[~result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x.svd_genome_recommender, x[1], 10), 1).mean(),
                                         result.loc[result.userId.isin(cold_users)].apply(lambda x: precision_at_k(x.svd_genome_recommender, x[1], 10), 1).mean()]
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.017540,0.271012
content_describtion_recommender,0.005014,0.007029,0.000000
content_soup_recommender,0.009655,0.013535,0.000000
content_genome_recommender,0.019762,0.027704,0.000000
content_genome_recommender_improved,0.033660,0.009860,0.092881
content_genome_popular_recommender,0.104898,0.038139,0.271012
svd_recommender,0.053018,0.018223,0.139595
svd_genome_recommender,0.061722,0.030426,0.139595


Мы видим, что точность рекомендаций улучшилась на "теплых" пользователях

### <a id="3">Сравнение моделей</a>

In [58]:
metrics

,full_precision,warm_precision,cold_precision
popular_recommender,0.090204,0.017540,0.271012
content_describtion_recommender,0.005014,0.007029,0.000000
content_soup_recommender,0.009655,0.013535,0.000000
content_genome_recommender,0.019762,0.027704,0.000000
content_genome_recommender_improved,0.033660,0.009860,0.092881
content_genome_popular_recommender,0.104898,0.038139,0.271012
svd_recommender,0.053018,0.018223,0.139595
svd_genome_recommender,0.061722,0.030426,0.139595


Я составила 8 разных рекомендательных систем. Из них для (content_genome_popular_recommender и svd_genome_recommender) являются гибридными, то есть вклячают логику друх разных алгоритмов.

Лучше всего на общем датасете показал алгоритм **content_genome_popular_recommender**.

С холодным стартом лучше всего справляется **popular_recommender**. Можно предположить, что новые пользователи предпочитают сначала закрыть общие потребности, то есть посмотреть популярные фильмы, которые любимы большинством. А потом уже смотреть фильмы, которые отражают индивидуальные предпочтения.

С "теплыми" пользователями лучше показал себя **content_genome_popular_recommender** и **svd_genome_recommender**. 

Дальнейшим шагом будет проверка рекомендательных систем на А/В тесте, где будет оцениваться их эффективность и положительный эффект. Если ресурсы позволяют, то можно проверить каждый из алгоритмов. Но если они ограничены, то можно проверить только те алгоритмы, которые получили больший показатель точности: **content_genome_popular_recommender, popular_recommender и svd_genome_recommender.**

##### Сохраняем данные и модель

In [79]:
popular_df.to_csv("final_data/popular_df.csv", index=False)
metrics.to_csv("final_data/metrics.csv")

In [80]:
with open("final_data/svd_model.pkl", "wb") as f:
    pickle.dump(svd_model, f)